In [1]:
import numpy as np
import pandas as pd
import datetime
from sqlalchemy import create_engine
import plotly.express as px
from database_utils import add_new_unl, add_new_dot, clean_unleashed, clean_dot, get_level_2

In [2]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [3]:
## PASTE 2 FILES BELOW AND UPDATE ALL TABLES

unl_download = r"C:\Users\mikej\Downloads\SalesEnquiryList - 2024-01-27T094953.271.xlsx"  ### file download from unleashed
dot_download = r"C:\Users\mikej\Downloads\SHOP Supplier Reporting - Invoice Details Excel - 2024-01-27T094939.310.xlsx"  ### file download from dot

add_new_unl(unl_download)  ## add rows to unleashed raw
add_new_dot(dot_download)  ## add rows to dot raw

clean_unleashed()    ### clean entire unleashed database
clean_dot()          ### clean entire dot database
get_level_2()        ### update complete level_2 db

In [ ]:
### updating all_sales_data.csv in "data" directory

level_2 = pd.read_sql("SELECT * FROM level_2 WHERE date > '2020-12-31';", con = engine)
# level_2 = level_2[level_2.market_segment != 'Samples']
level_2.to_csv(r"C:/Users/mikej/Desktop/cpg-sales/data/all_sales_data.csv", index=False)

In [4]:
### bar graph should populate if all is well

level_2 = pd.read_sql('SELECT * FROM level_2;', con = engine)
level_2.date = pd.to_datetime(level_2.date)

px.bar(level_2.set_index('date').groupby(pd.Grouper(freq='Q'))['usd'].sum(),
        y='usd',
        template='presentation',
        labels={'usd':'$USD',
                'date':''},
        color='usd',
        width=900,
        height=400)